In [5]:
import numpy as np
import numpy.linalg as la
import scipy.sparse as sp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
plt.style.use('bmh')

# Aufgabe 1

### a) Spannungsberechnung

In [6]:
def StressStrain(C, E):
    Emod = E.copy()
    Emod[3:] = (1/2)*Emod[3:]
    Sigma = C.dot(E)
    return Sigma

### b) Berechnung der Mises-Vergleichsspannung

In [7]:
def dev(A):
    voigt_eye = np.array([1,1,1,0,0,0])
    trA = np.sum(A*voigt_eye)
    d = A - (1/3)*trA*voigt_eye
    return d

def SigmaMises(S):
    devS = dev(S)
    SMises = np.sqrt(3/2)*devS.T.dot(devS)
    return SMises

### c) Steifigkeitsmatrizen in nicht-normierter Voigt Notation

In [8]:
def CISO(K, G):
    C = np.eye(6)
    C = G*C
    for i in [0,1,2]:
        C[i,i] = K + 4/3*G
    for (i,j) in [(0,1),(0,2),(1,2)]:
        C[i,j] = K - 2/3*G     
    C = np.maximum(C,C.T)
    return C

def CCUBIC(C1111, C1122, C2323):
    C = np.zeros([6,6])
    for (i,j) in [(0,0),(1,1),(2,2)]:
        C[i,j] = C1111
    for (i,j) in [(0,1),(0,2),(1,2)]:
        C[i,j] = C1122     
    for (i,j) in [(3,3),(4,4),(5,5)]:
        C[i,j] = C2323  
    C = np.maximum(C,C.T)
    return C


def CTISO(C1111, C3333, C2323, C1122, C1133):
    C = np.zeros([6,6])
    for (i,j) in [(0,0),(1,1)]:
        C[i,j] = C1111
    for (i,j) in [(0,1)]:
        C[i,j] = C1122
    for (i,j) in [(0,2),(1,2)]:
        C[i,j] = C1133
    for (i,j) in [(2,2)]:
        C[i,j] = C3333
    for (i,j) in [(3,3),(4,4)]:
        C[i,j] = C2323
    for (i,j) in [(5,5)]:
        C[i,j] = (C1111-C1122)/2 
    C = np.maximum(C,C.T)
    return C

In [9]:
def SpectralDecomposition( C ):
    """ C in nicht-normierter Voigt-Notation """
    
    # Basiswechsel
    Dsqrtt = np.array([ 1, 1, 1,  1/np.sqrt(2),  1/np.sqrt(2),  1/np.sqrt(2) ])
    Dsqrth = np.array([ 1, 1, 1,  np.sqrt(2),  np.sqrt(2),  np.sqrt(2) ])
    
    C_to_C2 = np.tile(Dsqrth,(6,1)).T*Dsqrth
    C2 = C * C_to_C2
    
    EW, EV = la.eig(C2)
    P = np.zeros([6,6,6])  ### Projektoren
    lambdas = np.array([])
    
    print(EW)
    
    for i in range(6):
        print("i: ", i)
        found = -1 ### Index, so dass  V(i,i) = lambda(found)

        ### Falls i = 1 -> nlambda = 1, lambda(nlambda) = V(1,1)
        if (len(lambdas) == 0): ### Ersten Eigenwert auf jeden Fall übernehmen
            lambdas = np.append(lambdas, EW[i])
            print("ersten EW übernommen ", EW[i])
            found = 0
        else:
            ### Falls schon mind. ein Eigenwert bearbeitet wurde:
            ### auf doppelte EW prüfen!
            for j in range(len(lambdas)):
                ### Falls 'numerisch gleich' Index speichern
                print("Comparing ", np.abs(lambdas[j]), " with ", str(1.e-8*la.norm(lambdas,ord=np.inf)))
                if ( np.abs(lambdas[j]) < 1.e-8*la.norm(lambdas,ord=np.inf)):
                    print(EW[i], "numerisch gleich wie ", j)
                    found = j
            if ( found == -1 ): ### Neuer Eigenwert gefunden
                print("neuer EW gefunden ", EW[i])
                lambdas = np.append(lambdas, EW[i])
                found = len(lambdas)
        print("found: ", found)
        print("lambdas", lambdas)
        ### u: Eigenvektor zu lambda(i)
        ### !!! ACHTUNG:  Die u_j zu _vielfachen Eigenwerten_ sind
        ### !!!           i.A. NICHT orthogonal -> Orthogonalisieren:
        u = EV[i]
        print("u: ", u)
        A = P[found][:][:] ### Projektor des EW (temporär)
        v = u - A*u; ### v = u - P_found u
        ### Eigenprojektor um v times v inkrementieren
        ### Die Division sorgt für die notwendige NORMIERUNG (||v|| = 1)
        A = A + v.dot(v.T)/(v.T.dot(v))
        ### Neuen Projektor abspeichern:
        P[found][:][:]=A
    ### Zurückrechnen von normierter Voigt-Notation nach ABQ-Notation
    #for i range(len(lambdas))
    #    P[i][:][:] =  .dot(np.squeeze(P[i][:][:])).dot()
    #end
    print("lambdas: ", lambdas)
    print("P: ", P)
    return P, lambdas, len(lambdas)

In [10]:
P, lambdas, nlambdas = SpectralDecomposition(CTISO(100000,110000, 120000, 115000, 112000))
C = np.zeros([6,6])
for i in range(nlambdas):
    C = C + lambdas[i]*(P[i])
print(C)
CTISO(100000,110000, 120000, 115000, 112000)

[ 329365.96417484  -15000.           -4365.96417484  240000.          240000.
  -15000.        ]
i:  0
ersten EW übernommen  329365.964175
found:  0
lambdas [ 329365.96417484]
u:  [-0.57328522 -0.70710678 -0.41393726  0.          0.          0.        ]
i:  1
Comparing  329365.964175  with  0.00329365964175
neuer EW gefunden  -15000.0
found:  2
lambdas [ 329365.96417484  -15000.        ]
u:  [-0.57328522  0.70710678 -0.41393726  0.          0.          0.        ]
i:  2
Comparing  329365.964175  with  0.00329365964175
Comparing  15000.0  with  0.00329365964175
neuer EW gefunden  -4365.96417484
found:  3
lambdas [ 329365.96417484  -15000.           -4365.96417484]
u:  [ -5.85395688e-01  -5.10866926e-16   8.10747734e-01   0.00000000e+00
   0.00000000e+00   0.00000000e+00]
i:  3
Comparing  329365.964175  with  0.00329365964175
Comparing  15000.0  with  0.00329365964175
Comparing  4365.96417484  with  0.00329365964175
neuer EW gefunden  240000.0
found:  4
lambdas [ 329365.96417484  -15000.

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:50: RuntimeWarning: divide by zero encountered in true_divide


IndexError: index 6 is out of bounds for axis 0 with size 6

In [ ]:
    lambdas, lambdas_indices = np.unique(EW.round(decimals=8), return_index=True)
    nlambdas = len(lambdas) # anzahl verschiedener EW
    
    eigenvectors = np.array([EV[:,i] for i in lambdas_indices])
    print (eigenvectors)
    

In [ ]:
def SpectralDecomposition( C ):
    """ C in nicht-normierter Voigt-Notation """
    
    # Basiswechsel
    Dsqrtt = np.array([ 1, 1, 1,  1/np.sqrt(2),  1/np.sqrt(2),  1/np.sqrt(2) ])
    Dsqrth = np.array([ 1, 1, 1,  np.sqrt(2),  np.sqrt(2),  np.sqrt(2) ])
    
    C_to_C2 = np.tile(Dsqrth,(6,1)).T*Dsqrth
    C2 = C * C_to_C2
    
    EW, EV = la.eig(C2)
    P = np.zeros([6,6,6])  ### Projektoren
    lambdas = np.array([])
    
    for i in range(6):
        found = 0 ### Index, so dass  V(i,i) = lambda(found)

        ### Falls i = 1 -> nlambda = 1, lambda(nlambda) = V(1,1)
        if (len(lambdas)): ### Ersten Eigenwert auf jeden Fall übernehmen
            np.append(lambdas, EW[i])
            found = 0
        else:
            ### Falls schon mind. ein Eigenwert bearbeitet wurde:
            ### auf doppelte EW prüfen!
            for j in range(len(lambdas)):
                ### Falls 'numerisch gleich' Index speichern
                if ( np.abs(lambdas[j]) < 1.e-8*norm(lambdas,Inf)):
                    found = j
            if ( found == 0 ): ### Neuer Eigenwert gefunden
                np.append(lambdas, EW[i])
                found = len(lambdas) + 1

        ### u: Eigenvektor zu lambda(i)
        ### !!! ACHTUNG:  Die u_j zu _vielfachen Eigenwerten_ sind
        ### !!!           i.A. NICHT orthogonal -> Orthogonalisieren:
        u = EV[i]
        A = P[found][:][:] ### Projektor des EW (temporär)
        v = u - A*u; ### v = u - P_found u
        ### Eigenprojektor um v times v inkrementieren
        ### Die Division sorgt für die notwendige NORMIERUNG (||v|| = 1)
        A = A + v.dot(v.T)/(v.T.dot(v))
        ### Neuen Projektor abspeichern:
        P[found][:][:]=A
    ### Zurückrechnen von normierter Voigt-Notation nach ABQ-Notation
    #for i range(len(lambdas))
    #    P[i][:][:] =  .dot(np.squeeze(P[i][:][:])).dot()
    #end
    print("i: ", i)
    print("lambdas: ", lambdas)
    print("P: ", P)
    return P, lambdas, len(lambdas)

In [ ]:
def SpectralDecomposition( C ):
    """ C in nicht-normierter Voigt-Notation """
    
    # Basiswechsel
    Dsqrtt = np.array([ 1, 1, 1,  1/np.sqrt(2),  1/np.sqrt(2),  1/np.sqrt(2) ])
    Dsqrth = np.array([ 1, 1, 1,  np.sqrt(2),  np.sqrt(2),  np.sqrt(2) ])
    
    C_to_C2 = np.tile(Dsqrth,(6,1)).T*Dsqrth
    P2_to_P = np.tile(Dsqrtt,(6,1)).T*Dsqrtt
    C2 = C * C_to_C2
    
    EW, EV = la.eig(C2)
    EW = EW.round(decimals=8)
    P = np.zeros([6,6,6])  ### Projektoren
    
    lambdas, lambdas_indices = np.unique(EW, return_index=True)
    nlambdas = len(lambdas) # anzahl verschiedener EW
    
    #print("EW: ", EW)
    #print("EV: ", EV)
    #print("lambdas: ", lambdas)
    
    
    P = np.zeros([nlambdas,6,6])
    for i in range(nlambdas):
        l = lambdas[i]
        idxes = np.where(EW == l)
        for idx in idxes[0]:
            u = EV[idx]
            A = P[i]
            v = u - A.dot(u)
            A = A + v.dot(v.T)/((v.T).dot(v))
            P[i] = A
    
    #print("P: ", P)
    P = P * P2_to_P
    return P, lambdas, len(lambdas)